<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/SingleSubjSVM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount = True)

Mounted at /content/gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source
!ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 1855, done.
remote: Counting objects: 100% (1855/1855), done.
remote: Compressing objects: 100% (1474/1474), done.
remote: Total 1855 (delta 1213), reused 718 (delta 364), pack-reused 0
Receiving objects: 100% (1855/1855), 93.50 MiB | 23.63 MiB/s, done.
Resolving deltas: 100% (1213/1213), done.
/content/teambrainiac/source
access_data.py			    process.py
brain_viz_single_subj.py	    single_subject.py
BuildSingleSubjectSVM_Models.ipynb  SingleSubjectSVM.ipynb
data				    streamlit
DL				    utils.py
group_svm			    VisualizationPlayground.ipynb
helper				    VisualizationsNotebook.ipynb
__init__.py


In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 228 bytes


In [11]:
# Import libraries
!pip install boto3 nilearn nibabel
#to load data
import pickle
#common packages we need
import numpy as np
import pandas as pd
#for brain imaging
import nibabel as nib
from nilearn import plotting, image
#for making graphs
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
%matplotlib inline
import seaborn as sns
from nilearn.image import threshold_img
#sklearn packages needed
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,auc,roc_auc_score,precision_recall_curve,precision_score,recall_score
from sklearn.svm import SVC
#import code for processing and visualizing data
from single_subject import *
from access_data import *
from brain_viz_single_subj import *

In [7]:
#get common variables we need across multiple cells
data_path_dict = get_data_dict()
subj_ids, _ = get_subj_information(data_path_dict)
_ , y_test,_= get_labels(data_path_dict)
##get brain images
affine_image,t1_image,mni_image = get_brain_templates()

In [ ]:
##setting mask list
#masks we want to run model on, needs to be nested list for cell to run
#indices of the masks we want 0 = whole brain mask and masks minus ROIs, 1 = ROIs
mask_indices = [0,1]
mask_list = []
# for ind in mask_indices:
#   mask_dict = get_mask_data(data_path_dict,ind)  
#   masks = list(mask_dict.keys())[3:]
#   mask_list.append(masks)
#Example of building your own
mask_dict = get_mask_data(data_path_dict,0)
mask_list = [['mask']]


In [8]:
#grab results
data_path = '/content/drive/My Drive/data/singlesubjectmodels/mask_tr_1_2_subject_models.pkl'

In [9]:
data = open_pickle(data_path)

In [21]:
labels = ['Decrease','Increase']

In [32]:
subj_id = subj_ids[0]
subj_data = data['mask'][subj_id]
subj_report = classification_report(y_test,subj_data['predicts']['run_03']['predicts'],target_names=labels,output_dict=True)

In [35]:
subj_df = pd.DataFrame(subj_report)
subj_df

,Decrease,Increase,accuracy,macro avg,weighted avg
precision,0.680851,0.729730,0.702381,0.705290,0.705290
recall,0.761905,0.642857,0.702381,0.702381,0.702381
f1-score,0.719101,0.683544,0.702381,0.701323,0.701323
support,42.000000,42.000000,0.702381,84.000000,84.000000


In [40]:
subj_id = subj_ids[0]

In [44]:
subj_predicts=data['mask'][subj_id]['predicts']
for run in list(subj_predicts.keys()):
  subjs_acc_list.append(run)
  subjs_acc_list.append(accuracy_score(y_test,subj_predicts[run]['predicts']))
  subjs_acc_list.append(accuracy_score(y_test[dec_trials],subj_predicts[run]['predicts'][dec_trials]))

In [45]:
subjs_acc_list

['1',
 'run_03',
 0.7023809523809523,
 0.35714285714285715,
 'run_04',
 0.8333333333333334,
 0.19047619047619047,
 'run_03',
 0.7023809523809523,
 0.7619047619047619,
 'run_04',
 0.8333333333333334,
 0.8571428571428571]

In [39]:
accuracy_scores_list = pd.DataFrame(columns=['subj_id','run','total_acc','acc_increase','acc_decrease','precision_all','precision_decrease','precision_increase',
                                    'recall_all','recall_increase','recall_decrease','f1_all','f1_increase','f1_decrease','auc_all','auc_increase','auc_decrease'])
dec_trials = np.where(y_test==0)
inc_trials = np.where(y_test==1)
for subj_id in subj_ids[0]:
  subjs_acc_list = [subj_id]
  subj_predicts=data['mask'][subj_id]['predicts']
  for run in list(subj_predicts.keys()):
    subjs_acc_list.append(run)
    subjs_acc_list.append(accuracy_score(y_test,subj_predicts[run]['predicts']))
    subjs_acc_list.append(accuracy_score(y_test[dec_trials],subj_predicts[run]['predicts'][inc_trials]))
  


KeyError: ignored

In [ ]:
data

In [16]:
subj_data

{'X_train': array([[ 0.        ,  0.        ,  0.        , ..., -0.35537407,
         -0.28912827, -0.15646337],
        [ 0.        ,  0.        ,  0.        , ..., -0.3566215 ,
         -0.29077935, -0.16021122],
        [ 0.        ,  0.        ,  0.        , ...,  2.8391807 ,
          2.9680855 ,  1.7116985 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  2.7306352 ,
          2.807277  ,  1.3825058 ],
        [ 0.        ,  0.        ,  0.        , ...,  2.780192  ,
          2.8530087 ,  1.4065295 ],
        [ 0.        ,  0.        ,  0.        , ...,  1.2044214 ,
          0.02133828, -0.35965276]], dtype=float32),
 'model': SVC(C=5, gamma='auto', probability=True),
 'predicts': {'run_03': {'decision_function': array([-0.18765468, -0.13211306,  0.04130328, -0.39873928, -0.46211708,
          -0.03996567, -0.0808285 , -0.06877901,  0.07424389,  0.21023356,
           0.06699925, -0.21993965, -0.4195237 , -0.02650632, -0.4374867 ,
          -0.178503  , -0.1